## Trump's January 6th Speech 
### A Slightly Less Basic Illustrative Example of Word2Vec

#### Introduction
This notebook builds off the Jupyter Notebook entitled "Introducing the Royal Family of Vectors". Much like its antecedent, it runs through another vectorization process but uses an original rather than a pre-processed text. I rely again on GenSim for the Word2Vec algorithm. The virtual environment to run this code is available on GitHub where one can also find the accompanying presentation, the other Jupyter Notebook and the text file used in this demonstration (Trump_JAN6.txt). The goal of this demonstration is to vectorize words from Trump's January 6th speech on the capital and to run very basic semantic similarity analysis.    

#### Getting Started- From raw text file to tokenization 
 The transcript from Trump January 6th speech can be found [here](https://www.npr.org/2021/02/10/966396848/read-trumps-jan-6-speech-a-key-part-of-impeachment-trial). I copied the transcript into a raw text file unaldutered, which I then named "Trump_JAN6.txt". As you may recall from the previous demonstration, the initial process to vectorize words from a given corpus consists of tokenizing the original text so they can be "fed" into the Word2Vec model. There are different ways to tokenize but in this case as in the previous, the goal is to a create a list of sentences where each sentence is a list of words (i.e., a list of lists). As before, the first step is to import the Word2Vec model from GenSim. I also import ```simple_preprocess``` from GenSim, which conviently converts a raw text file into tokens (see additional information [here](https://radimrehurek.com/gensim/utils.html)). Lastly, the module ```smart_open``` allows for the "reading in" of given textfiles and the module ```defaultdict```allows for the creation of dictionary-like objects (see lectures/homeworks for refresher). 

In [86]:
#Importing necessary modules and tokenizing original text
from gensim.models.word2vec import Word2Vec
from smart_open import smart_open
from gensim.utils import simple_preprocess
from gensim import corpora
from collections import defaultdict

tokens = [simple_preprocess(sentence, deacc=True) for sentence in open(r'Trump_JAN6.txt', encoding='utf-8')]

#### Weeding our tokens- Getting rid of stop words and unique tokens
Trump's speech contains a great number of stop words, a set of commonly used words (e.g., "for", "the", "and"),  which can "muddy" the model and subsequent semantic analysis. As such, we want to rid the speech of these stopwords and we want to make sure all our characters in each word is lowercase, which we achieve with the method ```.lower()```. Here under a define a list of stopwords (just a few for the purpose of the demonstration but one can add many more) that I then purge from the tokenized speech. 

In [89]:
#Creating a list of stop words and cleaning corpus of these stop words
stopwords = set('for a of the and to in who it'.split(' '))

cleaned_corpus = [[word.lower().strip() for word in sent if word not in stopwords] for sent in tokens]

As in the first demonstration, the speech also contains unique words that we want to purge from our training corpus to improve the model using ```defautldict```. The pythonic dictionnary will produce key/value pairs (in our cases, token/frequency pairs) that will render the exclusion of unique (i.e., rare) words easier from our corpus. 

In [100]:
# Creating a dictionary to rid the corpus of unique words
frequency = defaultdict(int)
for text in cleaned_corpus:
    for token in text:
        frequency[token] += 1
        
fully_cleaned_corpus = [[token for token in text if frequency[token] > 1] for text in cleaned_corpus]

It is worth noting that the cleaning of the corpus is relatively basic in this demonstration. Usually, in addition to purging both stopwords and rare words, one would want to lemmatize the raw corpus text. Lemmatization consists of grouping together inflected forms of words (e.g., change, changing and changed), which would further improve the model. 

#### Training the Word2Vec model

We are now ready to vectorize words using the Word2Vec algorithm in GenSim in exactly the same manner as in the previous demonstration. The model has been trained with a single speech and it is fair to assume our model is not very good for a robust semantic analysis (but hopefully sufficient to demonstrate the possibilities of Word2Vec). In practice, one would want to train a model on much larger corpora to truly obtain semantic accuracy (e.g., all of Trump's speeches during his presidency). Our model only contains 819 vectorized words (recall that our earlier model had a vocabulary of 100 698 words). 

In [106]:
#Training the Word2Vec model 
model_jan6=gensim.models.Word2Vec(fully_cleaned_corpus,min_count=2)

len(model_jan6.wv)

819

#### Using the Word2Vec for basic semantic analysis

Now that are model has been computed, lets examine some words. 

In [104]:
model_jan6.wv.most_similar(positive="fraud")

[('that', 0.9955363869667053),
 ('are', 0.9954153895378113),
 ('you', 0.995338499546051),
 ('they', 0.9952858686447144),
 ('is', 0.995254397392273),
 ('he', 0.9952491521835327),
 ('people', 0.9951519966125488),
 ('we', 0.9951363205909729),
 ('was', 0.9950646162033081),
 ('election', 0.9950599670410156)]

The word "election" is returned as one of the most similar semantically proximate words to the word "fraud". What if we compared "election" and "ballots" with some descriptors from his speech?   

In [112]:
model_jan6.wv.similarity('fraud', 'election')

0.9950598

In [113]:
model_jan6.wv.similarity('fair', 'election')

0.94566756

In [114]:
model_jan6.wv.similarity('fraud', 'ballots')

0.9948075

In [115]:
model_jan6.wv.similarity('fair', 'ballots')

0.9450196

In terms of similarity, the above pairings can be ranked from most similar to least similar as follows:  "fraud" & "ballots" > "fraud" & "election" > "fair" & "election" > "fair" & "ballots". We can tentatively conclude that, of the pairings, fraud and ballots are the most semantically proximate in Trump's January 6th speech. 

#### Conclusion 

This notebook has demonstrated how to vectorize words from a raw text file of Trump's January 6th speech using the Word2Vec algorithm within GenSim. It has additionally used the computed Word2Vec model to demonstrate basic semantic analysis. Applied work should use a much larger training corpora to truly optimize Word2Vec's ability for semantic analysis. Coding above was adapted from [this tutorial](https://radimrehurek.com/gensim/models/word2vec.html) and [this tutorial](https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-1/).